# 山富
- http://www.travel4u.com.tw/Web/Oversea/CountryPage.aspx?PageId=17

In [47]:
# 測試正規表達剔除字串
# http://www.isnowfy.com/regular-expression-negative/

sample= '\
<html>\
<body>\
<span> <a href="PageId=17">日本</a> </span>\
<span> <a href="PageId=60">東京</a> </span>\
<span> <a href="PageId=73#1116">日本遊輪</a> </span>\
</body>\
</html>'

import requests
from bs4 import BeautifulSoup
import re
soup = BeautifulSoup(sample, 'html.parser')
#print soup.select('span a')[0]

lands=[ l['href']for l in soup.select('span a') \
       if re.search('^(?!.*17)',l['href'] ) and re.search('(\d{2}$)',l['href'] ) ]    #.*:特定字串之前有任意字元任意個 ?!:不包含 ^:句首
print lands[0]


PageId=60


In [1]:
#日本館首頁-儲存日本各島連結

import requests
from bs4 import BeautifulSoup
res = requests.get('http://www.travel4u.com.tw/Web/Oversea/CountryPage.aspx?PageId=17',verify=False)
soup = BeautifulSoup(res.text, 'html.parser')
res.encoding = 'utf-8'
#print res.text
import re
import json

#List-lands
lands=[ "http://www.travel4u.com.tw"+l['href']for l in soup.select('#BodyContentPlaceHolder_KeyWordUserControl_KeyWordPanel span a') \
       if re.search('^(?!.*17)',l['href'] ) and re.search('(.*\d{2}$)',l['href'] ) ]
print lands[0]

http://www.travel4u.com.tw/Web/Oversea/CountryPage.aspx?PageId=60


In [8]:
#日本各島-儲存各島行程連結,再把各島行程連結匯總
#含重複連結！！！
'''
<div class=product_list>
    <div class=ss01> or <div class=aa01>
        <h1 href="...href...">title</h1>
        <div class=date>
            <span class=date_right> <a>privce</a> </span>
'''

hrefs=[]
prodNos=[]
titles=[] 
for i in xrange(1,len(lands)):
    land = lands[i]
    res = requests.get(lands[i],verify=False)
    soup = BeautifulSoup(res.text, 'html.parser')
    res.encoding = 'utf-8'
    
    '''儲存各島行程連結  
    hrefs=[ "http://www.travel4u.com.tw"+h['href'] for h in soup.select('.product_list h1 a') ]
    prodNos=[ str(hrefs[pn]).split('MGRUP_CD=')[-1] for pn in xrange(0,len(hrefs)) ]
    titles=[ t.text for t in soup.select('.product_list h1 a') ]
    print len(titles)''' 
    #使用.append將各島匯總
    [ hrefs.append("http://www.travel4u.com.tw"+h['href']) for h in soup.select('.product_list h1 a') ] #hrefs
    [ titles.append(t.text) for t in soup.select('.product_list h1 a') ] #titles
[ prodNos.append(str(hrefs[pN]).split('MGRUP_CD=')[1]) for pN in xrange(0,len(hrefs)) ] #prodNos

    
travel4u_jphrefs=[hrefs,prodNos,titles]
with open('/Users/min-go/documents-export-2016-05-27/travel4u_jphrefs','w') as f:
    json.dump(travel4u_jphrefs,f)
    
print len(hrefs)
print len(prodNos)
print len(titles)
print hrefs[1]
print prodNos[1]
print titles[1]

162
162
162
http://www.travel4u.com.tw/Oversea/explain_all.aspx?MGRUP_CD=TAK057768
TAK057768
浪漫姬路城.栗林公園.金刀比羅宮.淡路島夢舞台.抹茶體驗.環球影城哈利波特美食5日遊


In [100]:
#測試正規表達-用於id

sample2='\
<table id=GridView2>\
  <td class=gray2> <span id=GridView2_ctl02_Label11>2</span> </td>\
  <td class=gray2> <span id=GridView2_ctl02_Label12>...routes...</span> </td>\
</table>'

import requests
from bs4 import BeautifulSoup
soup = BeautifulSoup(sample2, 'html.parser')

days = [ int(d.text) for d in soup.select('#GridView2 td span') if re.search('GridView2_ctl(\d+)_Label11',d['id']) ]
print days[0]

2


In [124]:
#行程頁面-每日行程
'''
<table id=GridView2>  
    <tbody>
      <td class=gray2> <span id=GridView2_ctl02_Label11>2</span> </td> #第2天,Label11, ctl02開始(ctl01是班機資訊)
      <td class=gray2> <span id=GridView2_ctl02_Label12>...routes...</span> </td> 行程Label12
'''   

tour=[]
route={}
for j in xrange(0,len(hrefs)):
    res = requests.get(hrefs[j],verify=False)
    soup = BeautifulSoup(res.text, 'html.parser')
    days = [ d.text for d in soup.select('#GridView2 .gray2 span') if re.search('GridView2_ctl(\d+)_Label11',d['id']) ]
    locats = [ l.text for l in soup.select('#GridView2 .gray2 span') if re.search('GridView2_ctl(\d+)_Label12',l['id']) ]
    for j in xrange(0,len(days)):
        day = int(days[j])
        route[day]=locats[j]
#print route.values()[4] 
    routeTemp=route.copy()
    tour.append(routeTemp)

住宿飯店－大阪城公園(不上天守閣)～大阪水上巴士～免稅商店～心齋橋(或道頓堀自由逛街購物)～大阪～關西空港/台北


# 完整版

In [19]:
def gettour(url):
    route={}
#for j in xrange(0,len(hrefs)):
    res = requests.get(url,verify=False)
    soup = BeautifulSoup(res.text, 'html.parser')
    days = [ d.text for d in soup.select('#GridView2 .gray2 span') if re.search('GridView2_ctl(\d+)_Label11',d['id']) ]
    locats = [ l.text for l in soup.select('#GridView2 .gray2 span') if re.search('GridView2_ctl(\d+)_Label12',l['id']) ]
    for j in xrange(0,len(days)):
        day = "day"+days[j]
        route[day]=locats[j]
    return route

In [21]:
import requests
from bs4 import BeautifulSoup
res = requests.get('http://www.travel4u.com.tw/Web/Oversea/CountryPage.aspx?PageId=17',verify=False)
soup = BeautifulSoup(res.text, 'html.parser')
res.encoding = 'utf-8'
import re
import json

#List-lands-9筆
lands=[ "http://www.travel4u.com.tw"+l['href']for l in soup.select('#BodyContentPlaceHolder_KeyWordUserControl_KeyWordPanel span a') \
       if re.search('^(?!.*17)',l['href'] ) and re.search('(.*\d{2}$)',l['href'] ) ]

#日本各島-儲存各島行程連結,再把各島行程連結匯總-行程數:160筆
hrefs=[]
prodNos=[]
titles=[] 
for i in xrange(1,len(lands)):
    land = lands[i]
    res = requests.get(lands[i],verify=False)
    soup = BeautifulSoup(res.text, 'html.parser')
    res.encoding = 'utf-8'
    
    #使用.append將各島匯總
    [ hrefs.append("http://www.travel4u.com.tw"+h['href']) for h in soup.select('.product_list h1 a') ] #hrefs
    [ titles.append(t.text) for t in soup.select('.product_list h1 a') ] #titles
    [ prodNos.append(str(hrefs[pn]).split('MGRUP_CD=')[-1]) for pn in xrange(0,len(hrefs)) ] #prodNos

    
#一筆行程放入dic:pktour{}
#多筆行程放入list:pktours=[]
pktour={}
pktours=[]
for i in xrange(0,len(hrefs)):
    url= hrefs[i]
    routes = gettour(url)    #gettour(url) 回傳 route{} routes接住

    pktour['agency']='travel4u山富'
    pktour['prodNo']=prodNos[i]
    pktour['href']=hrefs[i]
    pktour['title']=titles[i]
    pktour['tour']=routes
    
    pktourTemp = pktour.copy()    #.copy(): pktour[]為參照位置,值不同丟入同一參照位值則值會更新只留最新一筆,故需要複製
    pktours.append(pktourTemp)    #使用.append放入list, 否則會出現IndexError: list assignment index out of range        

    
#輸出為.json
travel4u_jpPkTours={} 
travel4u_jpPkTours['travel4u']=pktours
with open('/Users/min-go/documents-export-2016-05-27/travel4u山富.json' , 'w') as f:
    json.dump(travel4u_jpPkTours, f)

SyntaxError: non-keyword arg after keyword arg (<ipython-input-21-b3ae85c01466>, line 50)

In [23]:
#匯入.json到Python
import json
with open('/Users/min-go/documents-export-2016-05-27/travel4u山富.json' , 'r') as f:
    travel4u = json.load(f)
    print type(travel4u)
    print travel4u.keys()[0]
    
#檢視.json的內容
print json.dumps(travel4u, encoding='UTF-8', ensure_ascii=False, sort_keys=True) 
#encoding='UTF-8'告訴JSON:travel4u裡面的編碼是'utf-8'

<type 'dict'>
travel4u
{"travel4u": [{"agency": "travel4u山富", "href": "http://www.travel4u.com.tw/Oversea/explain_all.aspx?MGRUP_CD=OSA05CI08X", "prodNo": "OSA05CI08X", "title": "特惠~京阪神姬路城清水寺倉敷美敷.哈利波特環球影城５日（高松進大阪出）", "tour": {"day1": "台北～高松空港～台北～四國香川～星空夜語", "day2": "住宿飯店－ 世界三大渦流～渦之道～海上散步～鳴門公園‧鳴門海峽－白牆街景 、古色古香 ．倉敷美觀地區〜★眺望世界文化遺產－姬路城(上天守閣)～京都", "day3": "住宿飯店～藝妓回憶錄拍攝場所~京都嵐山渡月橋－世界遺產‧京都千年名剎‧清水寺【米其林評鑑三星景點】～清水舞台‧音羽瀑布～地主神社‧姻緣石～國家傳統建物保存區‧石坂階梯‧二寧坂、產寧坂商店古街散策 －世界文化遺產～金閣寺(米其林三星景點)～大阪", "day4": "加贈哈利波特樂園入場卷大阪－全日暢遊歡樂的泉源【大阪環球影城】(一票到底 停留最久讓您玩的更盡興)－大阪", "day5": "住宿飯店～大阪城公園～免稅店～大阪流行指標區域：心齋橋、道頓堀、御堂筋、美國村、歐洲村商店地下街～關西空港～台北"}}, {"agency": "travel4u山富", "href": "http://www.travel4u.com.tw/Oversea/explain_all.aspx?MGRUP_CD=OSA05CI1099", "prodNo": "OSA05CI1099", "title": "特惠大阪樂遊~三都京阪神小火車.和服體驗.環球影城四項卷5日遊", "tour": {"day1": "台北／關西海上空港～前往心齋橋～道頓堀霓虹燈商店街～御堂筋大道或梅田商店地下街逛街(二擇一前往)", "day2": "大阪～環球影城(日本環球影城門票+哈利波特主題保證入場券+快速通關4項劵)～大阪", "day3": "住宿飯店～日式和服體驗～法國米其林三顆星～世界遺產．清水寺～清水舞台～地主神社～音羽之瀧～二、三年坂步道→世界遺產～金閣寺～陸舟之松→平安時代日本皇宮貴族渡假首選～